# Cantilever beam under gravity

```{admonition} Assemble a body force vector due to gravity
* define the body force vector

* linear-elastic analysis
```

The displacement due to gravity of a cantilever beam with young's modulus $E=206000$ N/mm$^2$, poisson ratio $\nu=0.3$, length $L=2000$ mm and cross section area $A=a \cdot a$ with $a=100$ mm is to be evaluated within a linear-elastic analysis.

![beam](images/beam.png)

First, let's create a meshed cube out of hexahedron cells with `n=(181, 9, 9)` points per axis. A numeric region created on the mesh represents the cantilever beam. A three-dimensional vector-valued displacement field is initiated on the region.

In [ ]:
import numpy as np
import felupe as fem

cube = fem.Cube(a=(0, 0, 0), b=(2000, 100, 100), n=(181, 9, 9))
region = fem.RegionHexahedron(cube)
displacement = fem.Field(region, dim=3)
field = fem.FieldContainer([displacement])

A fixed boundary condition is applied on the left end of the beam.

In [ ]:
boundaries = {"fixed": fem.dof.Boundary(displacement, fx=0)}

The material behaviour is defined through a built-in isotropic linear-elastic material formulation. The virtual work of internal forces is given by

$$ \delta W_{int} = -\int_v \delta \boldsymbol{\varepsilon} : \mathbb{C} : \boldsymbol{\varepsilon} ~ dv$$

with the linear elasticity tensor

$$ \mathbb{C} = 2\mu \boldsymbol{I} \odot \boldsymbol{I} + \lambda \boldsymbol{I} \otimes \boldsymbol{I}$$

and the stress tensor

$$ \boldsymbol{\sigma} = \mathbb{C} : \boldsymbol{\varepsilon} ~ .$$

In [ ]:
umat = fem.LinearElastic(E=206000, nu=0.3)
solid = fem.SolidBody(umat=umat, field=field)

The body force is defined by a (constant) gravity field on a solid body.

$$\delta W_{ext} = \int_v \delta \boldsymbol{u} \cdot \rho \boldsymbol{g} ~ dv$$

In [ ]:
gravity = fem.SolidBodyGravity(field, gravity=[0, 0, 9810], density=7850 * 1e-12)

Inside a Newton-Rhapson procedure, the weak form of linear elasticity is assembled into the stiffness matrix and the applied gravity field is assembled into the body force vector. The maximum displacement of the solution is identical to the one obtained in [[1]](https://www.doi.org/10.5545/sv-jme.2017.5081).

In [ ]:
step = fem.Step(items=[solid, gravity], boundaries=boundaries)
fem.Job(steps=[step]).evaluate()

A view on the field-container shows the deformed beam due to gravity.

In [ ]:
view = fem.View(field)
plotter = view.plot("Displacement", component=None, factor=500)
plotter.show(jupyter_backend="panel")

![beam-bodyforce](images/beam_bodyforce.png)

## References

[[1]](https://www.doi.org/10.5545/sv-jme.2017.5081) Glenk C. et al., *Consideration of Body Forces within Finite Element Analysis*, Strojniški vestnik - Journal of Mechanical Engineering, Faculty of Mechanical Engineering, 2018. [![doi](https://zenodo.org/badge/DOI/10.5545/sv-jme.2017.5081.svg)](https://www.doi.org/10.5545/sv-jme.2017.5081)